In [8]:
import numpy as np
import scipy.io as scio
import time
import onnxruntime
import adi

In [9]:
def GenerateSymbolMat(symbol_batch):
    _batch_size = symbol_batch.shape[0]
    _signal_dimension = symbol_batch.shape[1]
    _symbol_length = symbol_batch.shape[2]

    symbol_batch_real = np.real(symbol_batch)
    symbol_batch_imag = np.imag(symbol_batch)
    symbol_batch_mat = np.concatenate((symbol_batch_real,symbol_batch_imag), axis = 1).astype('float32')
    
    return symbol_batch_mat, _batch_size, _signal_dimension


# ZigBee
SymbolFilePath = './QPSK_Symbols.mat'
SymbolMat = 'QPSK_Symbols'

Symbol_file = scio.loadmat(SymbolFilePath)
Symbol_batch = Symbol_file[SymbolMat]
Symbol_batch = np.expand_dims(Symbol_batch,axis=0)


QPSK_Symbols, batch_size, signal_dimension = GenerateSymbolMat(Symbol_batch)
print(QPSK_Symbols.shape)
print(batch_size)
print(signal_dimension)

(1, 2, 672)
1
1


In [10]:
# Execution Provider of ONNX backend
EP_list = ['CPUExecutionProvider', 'CUDAExecutionProvider']
EPIdx = 1
print("Execution provider is", EP_list[EPIdx])
OQPSK_session = onnxruntime.InferenceSession("OQPSK_Modulator.onnx",providers=[EP_list[EPIdx]])

Execution provider is CUDAExecutionProvider


In [11]:
ONNX_Input = {OQPSK_session.get_inputs()[0].name: QPSK_Symbols}
ONNX_Output = OQPSK_session.run(None, ONNX_Input)
ONNX_Output_array = ONNX_Output[0]
print(ONNX_Output_array.shape)

(1, 2690, 2)


In [12]:
# Post processing for complex baseband
realSig = ONNX_Output_array[:,:,0]
imagSig = ONNX_Output_array[:,:,1]
cplxSig = realSig + 1j*imagSig
print(cplxSig.shape)

(1, 2690)


In [13]:
# Configure Pluto SDR
import iio
import adi
import time

sdr = adi.Pluto('ip:192.168.2.1') # or whatever your Pluto's IP is
centralFrequency = 2.405e9
samplingRate = 4e6
sdr.sample_rate = int(samplingRate) # sampling rate
sdr.tx_rf_bandwidth = int(samplingRate) # bandwidth
sdr.tx_lo = int(centralFrequency)
sdr.tx_hardwaregain_chan0 = 0 # Increase to increase tx power, valid range is -90 to 0 dB

In [14]:
# numFrames = 100
# txWaveform = cplxSig * 2**14 # Range mapping
# for pktIdx in range(batch_size):
#     samples = txWaveform[pktIdx,:]
#     for frameIdx in range(numFrames):
#         sdr.tx(samples)
#         time.sleep(0.01)


In [15]:
# Different message length
lengthRepeat = np.array([1,2,4,8]) # 16, 32,64,128 octes
SymLenIdx = 1
repeatFactor = lengthRepeat[SymLenIdx]
tmp_symbol = np.repeat(QPSK_Symbols,repeatFactor,axis=2)
tmp_ONNX_input = {OQPSK_session.get_inputs()[0].name: tmp_symbol}
tmp_ONNX_Output = OQPSK_session.run(None, tmp_ONNX_input)
tmp_ONNX_Output_array = tmp_ONNX_Output[0]
print(tmp_ONNX_Output_array.shape)
# Post processing for complex baseband
realSig = tmp_ONNX_Output_array[:,:,0]
imagSig = tmp_ONNX_Output_array[:,:,1]
cplxSig = realSig + 1j*imagSig
print(cplxSig.shape)

numFrames = 100
txWaveform = cplxSig * 2**14 # Range mapping
for pktIdx in range(batch_size):
    samples = txWaveform[pktIdx,:]
    for frameIdx in range(numFrames):
        sdr.tx(samples)
        time.sleep(0.02)
sdr.tx_destroy_buffer()

(1, 5378, 2)
(1, 5378)
